In [1]:
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()
    return text.split('\n')

In [3]:
def corpus_to_tokens_to_sequences(corpus,sequence_length=50):
    tokens = ' '.join(corpus)
    tokens = tokens.split()
    # organize into sequences of tokens
    length = sequence_length + 1
    sequences = []
    for i in range(length, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [5]:
clean_old_test = load_doc('/Users/tcbon/Desktop/Coding/Metis/Bootcamp/Project_4/Christianity - Bible/clean_old_testament.txt')

In [4]:
clean_new_test = load_doc('/Users/tcbon/Desktop/Coding/Metis/Bootcamp/Project_4/Christianity - Bible/clean_new_testament.txt')

In [7]:
# saving old testament sequences to txt file
old_test_sequences = corpus_to_tokens_to_sequences(clean_old_test)

resave_old_test = False
if resave_old_test == True:
    out_filename = 'old_test_sequences.txt'
    save_doc(sequences, out_filename)

Total Sequences: 609366


In [6]:
# saving new testament sequences to txt file
new_test_sequences = corpus_to_tokens_to_sequences(clean_new_test)

resave_new_test = False
if resave_new_test == True:
    out_filename = 'new_test_sequences.txt'
    save_doc(sequences, out_filename)

Total Sequences: 180528


### Old Testament Text Generator

In [8]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(old_test_sequences)
sequences = tokenizer.texts_to_sequences(old_test_sequences)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
old_test_model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_length),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

print(old_test_model.summary())
# compile model
old_test_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
old_test_model.fit(X, y, batch_size=128, epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            537550    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 10751)             1085851   
Total params: 1,774,301
Trainable params: 1,774,301
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1

KeyboardInterrupt: 

In [9]:
# # save the model to file
old_test_model.save('old_test_model_3.hdf5')
# save the tokenizer
pickle.dump(tokenizer, open('old_test_tokenizer_3.pkl', 'wb'))

### New Testament Text Generator

In [96]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(new_test_sequences)
sequences = tokenizer.texts_to_sequences(new_test_sequences)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_length),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 50)            300350    
_________________________________________________________________
lstm_8 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 6007)              606707    
Total params: 1,057,957
Trainable params: 1,057,957
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
180528/180528 [==============================] - 182s 1ms/sample - loss: 5.8808 - acc: 0.0820
Epoch 2/50
180528/1805

In [97]:
# save the model to file
model.save('new_test_model_2.hdf5')
# save the tokenizer
pickle.dump(tokenizer, open('new_test_tokenizer_2.pkl', 'wb'))